In [1]:
import pandas as pd
from pathlib import Path

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Dense,Embedding,Input,Activation,LSTM,GlobalMaxPool1D,Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import initializers, optimizers, layers
from sklearn.metrics import roc_auc_score
import re
import warnings
warnings.simplefilter(action="ignore")


2025-08-18 14:22:06.488708: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755526926.502149       8 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755526926.506131       8 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-18 14:22:06.521701: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
!ls /kaggle/input/jigsaw-toxic-comment-classification-challenge/


description.md				       test.csv
jigsaw-toxic-comment-classification-challenge  test.csv.zip
sample_submission.csv			       train.csv
sample_submission.csv.zip		       train.csv.zip


In [4]:
train_df = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip')
train_df.head()


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [5]:
test_df = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip')
test_df.head()


,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [6]:
train_df['comment_text'][0]


"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

In [7]:
pd.DataFrame(train_df[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].sum(),columns=['Count'])


,Count
toxic,15294
severe_toxic,1595
obscene,8449
threat,478
insult,7877
identity_hate,1405


In [8]:
temp = pd.DataFrame(train_df[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].sum(),columns=['Count'])


In [9]:
sns.barplot(temp.index, temp['Count'])


TypeError: barplot() takes from 0 to 1 positional arguments but 2 were given

In [10]:
from nltk import word_tokenize
train_df['tokenized_text'] = train_df.apply(lambda row: word_tokenize(row['comment_text']), axis=1)
lengths = [len(line) for line in train_df["tokenized_text"]]
train_df['comment_text'].iloc[np.argmax(lengths)]


'YOU!\nYou blocked me worm why!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [11]:
import plotly.express as px
px.histogram(lengths)


In [12]:
from nltk.corpus import stopwords
def process_text(data):
    stop = stopwords.words('english')
    data['processed_text'] = data.apply(lambda row: row['comment_text'].replace("\n"," "), axis=1) ## Remove new lines
    data['processed_text'] = data.apply(lambda row: re.sub('http://\S+|https://\S+', 'urls',row['processed_text']).lower(), axis=1) # Remove URL's
    data['processed_text'] = data.apply(lambda row: re.sub('[^A-Za-z ]+', '',row['processed_text']).lower(), axis=1) # Removes special characters, punctuations except alphabets
    data['processed_text'] = data['processed_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)])) # Removes Stop words
    data['processed_text'] = data.apply(lambda row: re.sub('  +', ' ',row['processed_text']).strip(), axis=1) # Removes extra spaces in between the words
    return data


In [13]:
train = process_text(train_df)
test = process_text(test_df)


In [14]:
train["processed_text"] = train.apply(lambda x: x["comment_text"] if len(x["processed_text"])==0 else x['processed_text'], axis=1)
test["processed_text"] = test.apply(lambda x: x["comment_text"] if len(x["processed_text"])==0 else x['processed_text'], axis=1)


In [15]:
num_words = 30000
tokenizer = Tokenizer(num_words=30000)
tokenizer.fit_on_texts(train['processed_text'])
train_tokens = tokenizer.texts_to_sequences(train['processed_text'])
test_tokens = tokenizer.texts_to_sequences(test['processed_text'])
train_seq = pad_sequences(train_tokens, maxlen=300)
test_seq = pad_sequences(test_tokens, maxlen=300)


In [16]:
print(train_seq.shape, test_seq.shape)


(159571, 300) (153164, 300)


In [17]:
train_labels = train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]]


In [18]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


In [19]:
filepath = '/kaggle/working/best_model_weights-{epoch:02d}.hdf5'
save_model_callback = ModelCheckpoint(filepath=filepath, monitor='val_auc', verbose=1,save_best_only=True, mode='max')


ValueError: The filepath provided must end in `.keras` (Keras model format). Received: filepath=/kaggle/working/best_model_weights-{epoch:02d}.hdf5

In [20]:
earlystop = EarlyStopping(monitor='val_auc', min_delta = 0.1, patience = 2, verbose = 1)


In [21]:
tf.keras.backend.clear_session()
input_layer = Input(shape = (300, ))
x = Embedding(30000, 200)(input_layer)
x = LSTM(60, return_sequences=True)(x)
x = GlobalMaxPool1D()(x)
x = Dropout(0.1)(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
output_layer = Dense(6, activation="sigmoid")(x)
model = Model(inputs=input_layer, outputs=output_layer)
model.summary()


2025-08-18 14:23:37.856009: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 300)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 300, 200)       │     6,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 300, 60)        │        62,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 60)             │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 60)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 50)             │         3,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           306 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,065,996 (23.14 MB)

 Trainable params: 6,065,996 (23.14 MB)

 Non-trainable params: 0 (0.00 B)

In [22]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy','AUC'])
model.fit(train_seq, train_labels, batch_size=128, validation_split=0.2, epochs = 5, callbacks=[save_model_callback, earlystop])


NameError: name 'save_model_callback' is not defined

In [23]:
sample_submission = pd.read_csv("/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip")


In [24]:
df_test = pd.merge(test, sample_submission, on = "id")


In [25]:
df_test.head()


,id,comment_text,processed_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...,yo bitch ja rule succesful youll ever whats ha...,0.5,0.5,0.5,0.5,0.5,0.5
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...,rfc title fine imo,0.5,0.5,0.5,0.5,0.5,0.5
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap...",sources zawe ashton lapland,0.5,0.5,0.5,0.5,0.5,0.5
3,00017563c3f7919a,":If you have a look back at the source, the in...",look back source information updated correct f...,0.5,0.5,0.5,0.5,0.5,0.5
4,00017695ad8997eb,I don't anonymously edit articles at all.,dont anonymously edit articles,0.5,0.5,0.5,0.5,0.5,0.5


In [26]:
y_pred = model.predict(test_seq)


4787/4787 ━━━━━━━━━━━━━━━━━━━━ 79s 16ms/step


In [27]:
df_test[["toxic","severe_toxic","obscene","threat","insult","identity_hate"]] = y_pred
df_test.head()


,id,comment_text,processed_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...,yo bitch ja rule succesful youll ever whats ha...,0.500577,0.503248,0.495192,0.502208,0.507097,0.499716
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...,rfc title fine imo,0.503102,0.508894,0.489976,0.509185,0.508447,0.500353
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap...",sources zawe ashton lapland,0.501933,0.507904,0.491014,0.509122,0.508598,0.499310
3,00017563c3f7919a,":If you have a look back at the source, the in...",look back source information updated correct f...,0.501088,0.509744,0.487890,0.509977,0.509674,0.501179
4,00017695ad8997eb,I don't anonymously edit articles at all.,dont anonymously edit articles,0.503002,0.508545,0.489399,0.510999,0.509831,0.500539


In [28]:
df_test.drop(["comment_text", "processed_text"], axis = 1, inplace = True)
df_test.to_csv("sample_submission.csv", index = False)
